# Тестовое задание для Realweb summer camp

Необходимо проанализировать следующие данные https://drive.google.com/file/d/16llNk3bEYLaS4ReJ8PBrXQFEehqOPf44/view

a.	Какой рекламный канал принёс больше всего дохода за всё время?  
b.	Как изменился средний чек транзакции после введения коронавирусных ограничений? Как он изменился для пользователей, покупающих с промокодом и без? (параметр «promo_activated»)  
c.	Можно ли с уверенностью в 95% сказать, что CR (коэффициент конверсии в транзакцию) в выходные дни отличается от CR в будние дни?  
d.	Вам необходимо спрогнозировать объем дохода, полученного с пользователей, приведенных на сайт контекстной рекламой (medium = cpc) на полгода вперед. 
Опишите, как бы вы подошли к этой задаче и какие дополнительные данные вам понадобятся?


In [76]:
#загружаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from scipy.stats import f_oneway


from datetime import datetime

In [166]:
#загружаем данные
path = 'd://realweb/s_c_data.csv'
df = pd.read_csv(path, encoding='cp1251')# в файле проблемы с кодировкой кириллицы

In [3]:
df.head()

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue
0,2020-05-11,google,organic,Не определено,Десктоп,no,no,3120,1233,5240,1048,525,90,86649.0
1,2020-05-11,yandex,cpc,Не определено,Мобайл,yes,no,3302,544,9930,1984,1416,217,244478.0
2,2020-05-11,google,cpc,Не определено,Мобайл,no,no,2970,1450,5460,1090,599,100,105150.0
3,2020-05-11,google,cpc,Не определено,Десктоп,no,no,1875,854,4250,848,407,71,79003.0
4,2020-05-11,yandex,organic,Не определено,Десктоп,no,no,2159,1000,4110,824,351,62,61861.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52724 entries, 0 to 52723
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                52724 non-null  object 
 1   source              52724 non-null  object 
 2   medium              52724 non-null  object 
 3   delivery_available  52724 non-null  object 
 4   device_type         52724 non-null  object 
 5   promo_activated     52724 non-null  object 
 6   filter_used         52724 non-null  object 
 7   pageviews           52724 non-null  int64  
 8   visits              52724 non-null  int64  
 9   productClick        52724 non-null  int64  
 10  addToCart           52724 non-null  int64  
 11  checkout            52724 non-null  int64  
 12  transactions        52724 non-null  int64  
 13  revenue             52724 non-null  float64
dtypes: float64(1), int64(6), object(7)
memory usage: 5.6+ MB


Нет пропущенных значений

In [5]:
df.describe()

,pageviews,visits,productClick,addToCart,checkout,transactions,revenue
count,52724.000000,52724.000000,52724.000000,52724.00000,52724.000000,52724.000000,5.272400e+04
mean,445.797853,127.219312,891.086981,178.04074,146.593620,20.092823,2.419841e+04
std,1108.633081,385.094298,2101.027285,419.99282,344.664158,52.436046,6.382118e+04
min,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000e+00
25%,12.000000,2.000000,20.000000,4.00000,3.000000,0.000000,0.000000e+00
50%,47.000000,7.000000,90.000000,18.00000,15.000000,2.000000,2.445500e+03
75%,256.000000,41.000000,520.000000,104.00000,83.000000,11.000000,1.329775e+04
max,26589.000000,6975.000000,32460.000000,6486.00000,3857.000000,1113.000000,1.397420e+06


In [6]:
df.shape

(52724, 14)

### Какой рекламный канал принёс больше всего дохода за всё время?

In [35]:
channel = df.groupby(['source'], as_index = False).sum()[['source', 'revenue']]# группируем по каналу и выводим суммарный доход
sorted_channel = channel.sort_values(by = ['revenue'], ascending=False)
sorted_channel[:3]

,source,revenue
21,yandex,4.674748e+08
12,google,4.565170e+08
0,(direct),2.384474e+08


Наибольший доход принесла реклама в Яндексе

### Как изменился средний чек транзакции после введения коронавирусных ограничений? Как он изменился для пользователей, покупающих с промокодом и без? (параметр «promo_activated»)

30 марта 2020 года были введены коронавирусные ограничения

In [43]:
df_before_lockdown = df[df['date']<='2020-03-30']#данные до ограничений
df_before_lockdown.shape

(16183, 14)

In [44]:
df_after_lockdown = df[df['date']>'2020-03-30']
df_after_lockdown.shape

(36541, 14)

In [45]:
df.shape[0] == df_before_lockdown.shape[0] + df_after_lockdown.shape[0]# проверим не потерялись ли данные

True

Данные не потерялись

In [46]:
#среднинй чек до введения
df_before_lockdown['checkout'].mean()

116.10307112401904

In [48]:
#средний чек после введения
df_after_lockdown['checkout'].mean()

160.09704167920967

Средний чек вырос

In [49]:
#без промокода до введения
df_before_lockdown[df_before_lockdown['promo_activated'] == 'no']['checkout'].mean()

89.62156278643447

In [51]:
#без промокода после введения
df_after_lockdown[df_after_lockdown['promo_activated'] == 'no']['checkout'].mean()

116.32619667428146

Средний чек вырос

In [50]:
#c промокодом до введения
df_before_lockdown[df_before_lockdown['promo_activated'] == 'yes']['checkout'].mean()

147.1065057008719

In [52]:
#с промокодом после введения
df_after_lockdown[df_after_lockdown['promo_activated'] == 'yes']['checkout'].mean()

206.59769725702677

Средний чек вырос

### Можно ли с уверенностью в 95% сказать, что CR (коэффициент конверсии в транзакцию) в выходные дни отличается от CR в будние дни?

In [167]:
# добавим рипзнак выходного дня
df['bus_day'] = df['date'].apply(np.datetime64)
df['bus_day'] = df['date'].apply(np.is_busday)

In [16]:
#добавим конверсию
df['conv'] = round(df['transactions']/df['visits'],4)

In [17]:
df.head(3)

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,bus_day,conv
0,2020-05-11,google,organic,Не определено,Десктоп,no,no,3120,1233,5240,1048,525,90,86649.0,True,0.0730
1,2020-05-11,yandex,cpc,Не определено,Мобайл,yes,no,3302,544,9930,1984,1416,217,244478.0,True,0.3989
2,2020-05-11,google,cpc,Не определено,Мобайл,no,no,2970,1450,5460,1090,599,100,105150.0,True,0.0690


In [18]:
# выделим выходные
df_weekend = df[df['bus_day'] == False]
#выделим будни
df_busdays = df[df['bus_day'] == True]

In [19]:
df.shape[0] == df_weekend.shape[0] + df_busdays.shape[0]

True

In [21]:
#оценим отличие между конверсией по выходным и конверсией по будням
f_oneway(df_weekend['conv'], df_busdays['conv'])

F_onewayResult(statistic=41.80561164114753, pvalue=1.0169205948811639e-10)

p-уровень значимости равен 10^(-10), очень маленькое значение, значит мы можем считать, что уровень конверсии в будние дни и в выходные отличается

### Вам необходимо спрогнозировать объем дохода, полученного с пользователей, приведенных на сайт контекстной рекламой (medium = cpc) на полгода вперед. Опишите, как бы вы подошли к этой задаче и какие дополнительные данные вам понадобятся?

Введем доп признак - количество дней от начальной даты и построим уравнение регрессии. Потом посчитаем прогноз на каждый день (в ближайшие полгода) и просуммируем полученные величины. В таком подходе также необходимо спрогнозировать (или получить каким-либо другим способом) и значения других переменных на рассматриваемый период.

In [26]:
#выбросим столбцы с конверсией и датой
df = df.drop(['conv'],axis = 1)

In [168]:
#выделим пользователей с medium = cpc
df = df[df['medium'] == 'cpc']

In [169]:
df.shape

(16835, 15)

In [170]:
#заменим все yes\no на 1\0
df['promo_activated'].replace(['no', 'yes'], [0, 1], inplace=True)
df['filter_used'].replace(['no', 'yes'], [0, 1], inplace=True)
df = df.drop(['medium'],axis=1)# значение для всех записей одинаковое, выбросим его
df.head(3)

,date,source,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,bus_day
1,2020-05-11,yandex,Не определено,Мобайл,1,0,3302,544,9930,1984,1416,217,244478.0,True
2,2020-05-11,google,Не определено,Мобайл,0,0,2970,1450,5460,1090,599,100,105150.0,True
3,2020-05-11,google,Не определено,Десктоп,0,0,1875,854,4250,848,407,71,79003.0,True


In [124]:
#функция возвращает месяц из даты
def get_delta(date):
    start = '2020-01-01'#начальная дата
    datee = np.datetime64(date)-np.datetime64(start)
    days = datee.astype('timedelta64[D]')
        
    return days / np.timedelta64(1, 'D')

In [171]:
df['days'] = df['date'].apply(get_delta)#добавили столбец с месяцем
df.head(2)

,date,source,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,bus_day,days
1,2020-05-11,yandex,Не определено,Мобайл,1,0,3302,544,9930,1984,1416,217,244478.0,True,131.0
2,2020-05-11,google,Не определено,Мобайл,0,0,2970,1450,5460,1090,599,100,105150.0,True,131.0


In [172]:
#столбец date будет коррелировать с day, уберем его
df = df.drop(['date'],axis=1)

In [173]:
cat_col = df.select_dtypes(include=['object']).columns
cols = list(cat_col)# выделяем категориальные признаки

In [174]:
cols_int = list(df.drop(columns = cols).columns)#убираем категориальные признаки

In [175]:
#Отделим целевую переменную
target = df['revenue']
data = df.drop(['revenue'],axis=1)

In [176]:
data.corr()

,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,bus_day,days
promo_activated,1.000000,0.008291,0.064339,-0.109955,0.138916,0.139022,0.172609,0.132774,-0.005632,0.027482
filter_used,0.008291,1.000000,-0.255963,-0.226926,-0.267147,-0.267102,-0.270250,-0.278234,-0.005195,-0.016014
pageviews,0.064339,-0.255963,1.000000,0.810638,0.865324,0.865284,0.822870,0.809484,-0.041936,0.033192
visits,-0.109955,-0.226926,0.810638,1.000000,0.704965,0.704869,0.582812,0.615521,-0.023136,-0.035624
productClick,0.138916,-0.267147,0.865324,0.704965,1.000000,0.999998,0.879474,0.961908,-0.033724,-0.051729
addToCart,0.139022,-0.267102,0.865284,0.704869,0.999998,1.000000,0.879532,0.961966,-0.033712,-0.051860
checkout,0.172609,-0.270250,0.822870,0.582812,0.879474,0.879532,1.000000,0.886788,-0.053511,0.054273
transactions,0.132774,-0.278234,0.809484,0.615521,0.961908,0.961966,0.886788,1.000000,-0.060944,-0.085240
bus_day,-0.005632,-0.005195,-0.041936,-0.023136,-0.033724,-0.033712,-0.053511,-0.060944,1.000000,0.007128
days,0.027482,-0.016014,0.033192,-0.035624,-0.051729,-0.051860,0.054273,-0.085240,0.007128,1.000000


Сильная корреляция между productClick, addToCart, pageviews, checkout, transactions


In [177]:
data = data.drop(['productClick','addToCart','pageviews','checkout'], axis=1)
data.corr()

,promo_activated,filter_used,visits,transactions,bus_day,days
promo_activated,1.000000,0.008291,-0.109955,0.132774,-0.005632,0.027482
filter_used,0.008291,1.000000,-0.226926,-0.278234,-0.005195,-0.016014
visits,-0.109955,-0.226926,1.000000,0.615521,-0.023136,-0.035624
transactions,0.132774,-0.278234,0.615521,1.000000,-0.060944,-0.085240
bus_day,-0.005632,-0.005195,-0.023136,-0.060944,1.000000,0.007128
days,0.027482,-0.016014,-0.035624,-0.085240,0.007128,1.000000


In [178]:
#кодируем не числовые переменные
data_col = data[cat_col]
data_col_dummies = pd.get_dummies(data_col)

In [179]:
#убираем категориальные переменные
data_wo_cat = data.drop(cat_col, axis=1)

In [180]:
#сольем все в один набор
concat = np.concatenate((data_col_dummies,data_wo_cat), axis=1)
final_data = pd.DataFrame(concat)
final_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,1,0,544,217,True,131.0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1450,100,True,131.0
2,0,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,854,71,True,131.0
3,0,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,1441,76,True,131.0
4,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,1842,150,True,131.0


In [181]:
#отмасштабируем признаки
scaler = StandardScaler()
scaler.fit(final_data)
scaled_data = scaler.transform(final_data)
scaled_data = pd.DataFrame(scaled_data)
scaled_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.504451,-0.054029,-0.710259,-0.114804,-0.090245,-0.249826,-0.10308,1.342422,-0.080727,-0.072489,...,0.892254,-0.859740,1.048892,-0.331184,1.061211,-0.699874,0.990137,4.420485,0.641116,-0.094274
1,-0.504451,-0.054029,1.407937,-0.114804,-0.090245,-0.249826,-0.10308,-0.744922,-0.080727,-0.072489,...,0.892254,-0.859740,1.048892,-0.331184,-0.942320,-0.699874,3.219437,1.781924,0.641116,-0.094274
2,-0.504451,-0.054029,1.407937,-0.114804,-0.090245,-0.249826,-0.10308,-0.744922,-0.080727,-0.072489,...,0.892254,1.163142,-0.953387,-0.331184,-0.942320,-0.699874,1.752922,1.127922,0.641116,-0.094274
3,-0.504451,-0.054029,-0.710259,-0.114804,-0.090245,-0.249826,-0.10308,1.342422,-0.080727,-0.072489,...,0.892254,1.163142,-0.953387,-0.331184,-0.942320,-0.699874,3.197292,1.240681,0.641116,-0.094274
4,-0.504451,-0.054029,-0.710259,-0.114804,-0.090245,-0.249826,-0.10308,1.342422,-0.080727,-0.072489,...,0.892254,-0.859740,1.048892,-0.331184,-0.942320,-0.699874,4.183991,2.909514,0.641116,-0.094274


#### Построение моделей

In [182]:
#Линейная регрессия
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(scaled_data, target, test_size = 0.3, random_state = 42)

In [183]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [184]:
r2 = r2_score(y_test, predictions)
r2

0.956512062988181

In [185]:
#Гребневая регуляризация
model = Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
r2

0.9565059292252687

In [186]:
#Лассо регуляризация 
model = Lasso()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
r2

0.956508524618261

Победила линейная регрессия